In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dmyplant2
dmyplant2.cred()
mp = dmyplant2.MyPlant(3600)

In [2]:
dval = dmyplant2.Validation.load_def_csv("input2.csv")
dval;

In [3]:
i = 5
vale = dval.iloc[i][['serialNumber','Validation Engine','val start']]

In [4]:
e=dmyplant2.Engine.from_sn(mp,vale['serialNumber'], valstart=vale['val start'])
pd.DataFrame.from_dict(e.dash, orient='index').T

,Name,Engine ID,Design Number,Engine Type,Engine Version,P,P_nom,BMEP,serialNumber,id,Count_OpHour,val start,oph@start,oph parts,LOC
0,BMW Regensburg,M2,I448,616,J01,16,2744.9,22.0,1243362,157701,71738.0,2020-09-07,0,71738,None


In [5]:
#e.get_messages()

In [6]:
%reload_ext autoreload
%autoreload 2
from dfsm import msgFSM
fsm = msgFSM(e)
fsm.run() # run the Finite State Machine
fsm.store()

Scan Messages: 100%|████████████████████████████████████████████████████| 31796/31796 [00:02<00:00, 12667.06 messages/s]


In [7]:
%reload_ext autoreload
%autoreload 2
from dfsm import FSM
FSM.dot('FSM.dot')

In [9]:
fsm._runlog

['15.09.2020 12:15:53 |standstill              0.0s    0.0s 1231 Request module on                          1   0  on MANUAL => start-preparation   ',
 '15.09.2020 12:21:50 |start-preparation     356.5s  356.5s 1249 Starter on                                 1   0  on MANUAL => starter             ',
 '15.09.2020 12:21:54 |starter                 4.7s  361.3s 3225 Ignition on                                1   0  on MANUAL => hochlauf            ',
 '15.09.2020 12:22:35 |hochlauf               40.9s  402.2s 2124 Idle                                       1   0  on MANUAL => idle                ',
 '15.09.2020 12:23:10 |idle                   34.3s    0.0s 3226 Ignition off                               1   0 off  OFF => standstill          ',
 '15.09.2020 12:34:44 |standstill            694.3s    0.0s 1231 Request module on                          2   0  on MANUAL => start-preparation   ',
 '15.09.2020 12:40:59 |start-preparation     375.2s  375.2s 1249 Starter on                     

In [10]:
rdf = pd.DataFrame(fsm._starts)
rda = rdf[['success','mode','cumtime','start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation']]
#rda = rdf[((rdf['mode'] == 'AUTO') & (rdf['success'] == True))]
#rda = rdf[((rdf['mode'] == 'AUTO') & (rdf['success'] == True) & (rdf['synchronize'] < 300.0) & (rdf['start-preparation'] < 300.0))]
#rda = rdf[((rdf['success'] == True) & (rdf['start-preparation'] < 1000.0))]
#rda = rdf[((rdf['success'] == True))]
rda.round(2)

,success,mode,cumtime,start-preparation,starter,hochlauf,idle,synchronize,load-ramp
0,False,MANUAL,436.57,356.55,4.75,40.94,34.33,NaN,NaN
1,False,MANUAL,405.37,375.22,4.43,25.72,NaN,NaN,NaN
2,False,MANUAL,1090.13,1090.13,NaN,NaN,NaN,NaN,NaN
3,True,MANUAL,9162.22,197.15,4.74,22.43,47.56,101.15,158.73
4,True,MANUAL,541.16,186.92,4.64,25.01,42.74,15.64,158.73
...,...,...,...,...,...,...,...,...,...
345,True,AUTO,30741.91,96.24,4.24,26.66,58.78,23.20,158.73
346,False,MANUAL,70.85,70.85,NaN,NaN,NaN,NaN,NaN
347,False,MANUAL,77.27,77.27,NaN,NaN,NaN,NaN,NaN
348,True,AUTO,54594.36,64.96,5.16,20.30,58.90,56.98,158.73


In [ ]:
rda.describe().round(2).T

In [ ]:
rda[['start-preparation','starter','hochlauf','idle','synchronize','cumtime']].hist(figsize = (20,12), bins=100);
#rda.hist(figsize = (20,12), bins=100);

In [ ]:
rda.plot.box(subplots=True, grid = True, figsize=(24,10), layout=(1,7));

In [ ]:
rda.plot.box(subplots=True, grid = True, figsize=(24,10), layout=(1,7), sym='');

In [ ]:
round((rdf[rdf.success].shape[0] / rdf.shape[0]) * 100,1)

In [ ]:
for i,c in rdf.iterrows():
    if len(c['alarms']) > 0:
        print(f"\nStartversuch: {i}, Success: {c['success']}")
        for a in c['alarms']:
            print(f"{c['mode']:15} {a['state']:20} {pd.to_datetime(int(a['msg']['timestamp'])*1e6).strftime('%d.%m.%Y %H:%M:%S')} {a['msg']['name']} {a['msg']['message']}")



In [ ]:
phases = [['start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation'],'start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation']
fig = plt.figure();
color = 'red'

for phase in phases:
    al = fsm.alarms_pareto(phase)[:20]
    if not al.empty:
        al['msg'] = al['msg'] + ' (' + al['name'] + ')'
        al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title=' | '.join(phase) if type(phase) == list else phase);
        plt.plot();


In [ ]:
phases = [['start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation'],'start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation']
fig = plt.figure();
color = 'orange'

for phase in phases:
    al = fsm.warnings_pareto(phase)[:20]
    if not al.empty:
        al['msg'] = al['msg'] + ' (' + al['name'] + ')'
        al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title=' | '.join(phase) if type(phase) == list else phase);
        plt.plot();


In [ ]:
#fsm.save_messages('msg4.txt')

In [ ]:
#fsm.store()